In [1]:
import pandas as pd 
import geopandas as gp
from shapely.ops import nearest_points
from shapely.geometry import Point
import numpy as np
from scipy.spatial import cKDTree

In [2]:
path = r'/home/semipro321/Documents/Workspace/Geospatial-Analysis-on-House-Price-Prediction/Inputs/'

In [3]:
df_house = pd.read_csv(path + 'House/house_with_neigh.csv')
df_school = pd.read_csv(path + 'City/Schools_with_neigh.csv')
gpf_house = gp.GeoDataFrame(df_house, crs="EPSG:4326", geometry=gp.points_from_xy(df_house.long,df_house.lat))
gpf_school= gp.GeoDataFrame(df_school, crs="EPSG:4326", geometry=gp.points_from_xy(df_school.LONGITUDE,df_school.LATITUDE))


I think this is wrong

In [4]:
# def near(df, col: [], point, pts):
#      '''
#      df := pass in the df you want to search
#      col := the column you want to return (LISt)
#      point := the geometry of left dataset
#      pts := union of dataset
#      '''
#      nearest = df.geometry == nearest_points(point, pts)[1]
#      return df[nearest][col].values[0]


# nearestschools = gpf_house.apply(lambda row: near(gpf_school, ['_id', 'NAME'], row.geometry, gpf_school.geometry.unary_union), axis=1)

In [5]:
def ckdnearest(gdA, gdB, distcol):
    gdA['geometry'] = gdA.geometry.to_crs('EPSG:5234')
    gdB['geometry'] = gdB.geometry.to_crs('EPSG:5234')

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    #If you want other features from B
    #gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            #If you want other features from B
            #gdB_nearest,
            pd.Series(dist, name=distcol)
        ], 
        axis=1)

    return gdf

In [6]:
# def calcdistance(origindf, destdf):
#     origindf = origindf['geometry'].to_crs('EPSG:5234')
#     destdf = destdf['geometry'].to_crs('EPSG:5234')
#     return origindf.distance(destdf)
# gpf_house['distance_nearest_school'] = calcdistance(gpf_house, mergegp)

In [7]:
gpf_house = ckdnearest(gpf_house, gpf_school, 'dist_nearest_school' )

In [8]:
gpf_house

,lat,long,sqft,parking,mean_district_income,bedrooms_bg,bedrooms_ag,bathrooms,final_price,type_Att/Row/Twnhouse,...,type_Condo Apt,type_Condo Townhouse,type_Detached,type_Link,type_Plex,type_Semi-Detached,type_Store W/Apt/Offc,geometry,neighbourhood,dist_nearest_school
0,43.661896,-79.385748,850.0,1,56526,1,2,2,855000,0,...,1,0,0,0,0,0,0,POINT (-1401264.376 14395931.334),Bay Street Corridor,211.820254
1,43.646982,-79.530356,NaN,6,52787,0,3,2,885000,0,...,0,0,0,0,0,1,0,POINT (-1390000.672 14400486.857),Islington-City Centre West,208.428213
2,43.708472,-79.397498,550.0,0,57039,0,1,1,550000,0,...,1,0,0,0,0,0,0,POINT (-1399025.711 14390988.757),Mount Pleasant West,164.834991
3,43.638399,-79.414448,650.0,1,70623,1,1,1,665000,0,...,1,0,0,0,0,0,0,POINT (-1399597.365 14399120.898),Niagara,817.632077
4,43.597096,-79.509612,NaN,1,44101,0,2,2,825513,0,...,0,0,1,0,0,0,0,POINT (-1393047.587 14405620.198),New Toronto,256.452172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15229,43.648317,-79.393065,750.0,1,70600,0,2,2,785000,0,...,1,0,0,0,0,0,0,POINT (-1401049.699 14397588.393),Waterfront Communities-The Island,259.294262
15230,43.706338,-79.262789,3250.0,2,36232,0,5,5,1025000,0,...,0,0,1,0,0,0,0,POINT (-1409950.195 14388508.316),Clairlea-Birchmount,373.825744
15231,43.788258,-79.160342,NaN,1,54045,0,3,3,681000,1,...,0,0,0,0,0,0,0,POINT (-1415912.440 14377320.780),Centennial Scarborough,4046.387302
15232,43.660846,-79.378756,550.0,0,53583,0,1,1,590000,0,...,1,0,0,0,0,0,0,POINT (-1401857.924 14395907.377),Church-Yonge Corridor,108.606713


In [9]:
df_ttc = pd.read_csv(path + 'Transit/Stops.csv')
gpf_ttc = gp.GeoDataFrame(df_ttc, crs="EPSG:4326", geometry=gp.points_from_xy(df_ttc.stop_lon,df_ttc.stop_lat))
gpf_house = ckdnearest(gpf_house, gpf_ttc, 'dist_nearest_ttc')

In [10]:
# nearestttc = gpf_house.apply(lambda row: near(gpf_ttc, ['stop_id'], row.geometry, gpf_ttc.geometry.unary_union), axis=1)
# nearestttc = nearestttc.apply(pd.Series)
# nearestttc.columns = ['stop_id']
# mergegpttc = gpf_ttc.merge(nearestttc, on='stop_id', how='right')

In [11]:
gpf_house

,lat,long,sqft,parking,mean_district_income,bedrooms_bg,bedrooms_ag,bathrooms,final_price,type_Att/Row/Twnhouse,...,type_Condo Townhouse,type_Detached,type_Link,type_Plex,type_Semi-Detached,type_Store W/Apt/Offc,geometry,neighbourhood,dist_nearest_school,dist_nearest_ttc
0,43.661896,-79.385748,850.0,1,56526,1,2,2,855000,0,...,0,0,0,0,0,0,POINT (-1401264.376 14395931.334),Bay Street Corridor,211.820254,1961.183353
1,43.646982,-79.530356,NaN,6,52787,0,3,2,885000,0,...,0,0,0,0,1,0,POINT (-1390000.672 14400486.857),Islington-City Centre West,208.428213,1250.003079
2,43.708472,-79.397498,550.0,0,57039,0,1,1,550000,0,...,0,0,0,0,0,0,POINT (-1399025.711 14390988.757),Mount Pleasant West,164.834991,4260.161880
3,43.638399,-79.414448,650.0,1,70623,1,1,1,665000,0,...,0,0,0,0,0,0,POINT (-1399597.365 14399120.898),Niagara,817.632077,495.583676
4,43.597096,-79.509612,NaN,1,44101,0,2,2,825513,0,...,0,1,0,0,0,0,POINT (-1393047.587 14405620.198),New Toronto,256.452172,2422.597469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15229,43.648317,-79.393065,750.0,1,70600,0,2,2,785000,0,...,0,0,0,0,0,0,POINT (-1401049.699 14397588.393),Waterfront Communities-The Island,259.294262,1097.259214
15230,43.706338,-79.262789,3250.0,2,36232,0,5,5,1025000,0,...,0,1,0,0,0,0,POINT (-1409950.195 14388508.316),Clairlea-Birchmount,373.825744,1388.667688
15231,43.788258,-79.160342,NaN,1,54045,0,3,3,681000,1,...,0,0,0,0,0,0,POINT (-1415912.440 14377320.780),Centennial Scarborough,4046.387302,5718.565912
15232,43.660846,-79.378756,550.0,0,53583,0,1,1,590000,0,...,0,0,0,0,0,0,POINT (-1401857.924 14395907.377),Church-Yonge Corridor,108.606713,1800.128917
